In [60]:
data = {
    '研究目的': [
        'Commercial batch\n商业化批次', 
        'Tris HCl change\n盐酸三羟甲基氨基甲烷变更', 
        'DS Suite 2 Line 2 PV batches\n新增原液车间2原液2号生产线PV批次',
        'Sodium acetate trihydrate & L-glutamic acid supplier change\n醋酸钠三水合物和谷氨酰胺变更',
        'WCB5 Cell Bank validation\n新细胞库WCB5 验证批',
        'DS Suite 2 Line 1 PV batches\n新增原液车间2原液1号生产线PV批次',
        'Commercial batch\n商业化批次',
        'DS Suite 2 Line 2 PV batches\n新增原液车间2原液2号生产线PV批次',
        'On-going stability study of 2022\n2022年持续稳定性研究',
        'Schott vial PV batch\n肖特西林瓶变更PV批次',
        'DP filling batch size enlarge\nDP扩批',
        'DS Suite 2 Line 1 PV batches\n新增原液车间2原液1号生产线PV批次',
        
    ],
    'Batch No.\n批号': [
        '1000002164,1000002165,1000002169',
        '1000004187', 
        '1000004606,1000004607,1000004608,1000004609,1000004611,1000007683',
        '1000009585',
        '1000010490',
        '1000013150,1000013152,1000013154,1000013156,1000013158,1000013160',
        '1000002930,1000002931,1000002932',
        '1000006705,1000006706,1000006707,1000006708',
        'G202201001',
        '1000007310,1000007311,1000007312',
        '1000010494,1000010495,1000010496',
        '1000014839,1000014840,1000014841,1000014842'
        
        
    ],
    'type': [
        'BDS stability study\n原液稳定性研究目的', 
        'BDS stability study\n原液稳定性研究目的', 
        'BDS stability study\n原液稳定性研究目的',
        'BDS stability study\n原液稳定性研究目的',
        'BDS stability study\n原液稳定性研究目的',
        'BDS stability study\n原液稳定性研究目的',
        'DP stability study\n成品稳定性研究目的',
        'DP stability study\n成品稳定性研究目的',
        'DP stability study\n成品稳定性研究目的',
        'DP stability study\n成品稳定性研究目的',
        'DP stability study\n成品稳定性研究目的',
        'DP stability study\n成品稳定性研究目的',
        # ... more items
    ]
}
df = pd.DataFrame(data)

In [81]:
import io
import pandas as pd
from docx import Document
import ipywidgets as widgets
from IPython.display import display, clear_output


# Upload widget for Excel files
uploader = widgets.FileUpload(accept='.xlsx', multiple=False)

# Function to capture changes from widgets directly into the DataFrame
def on_text_change(change, row, col):
    df.at[row, col] = change.new

def create_interactive_table(df):
    output = widgets.Output()
    
    def redraw_table():
        with output:
            clear_output(wait=True)
            for i in df.index:
                for col in df.columns:
                    text_widget = widgets.Text(value=str(df.at[i, col]), description=col)
                    text_widget.observe(lambda change, row=i, col=col: on_text_change(change, row, col), names='value')
                    display(text_widget)
    
    def add_row(b):
        df.loc[len(df)] = ['' for _ in range(len(df.columns))]
        redraw_table()
    
    def delete_row(b):
        if len(df) > 0:
            df.drop(df.tail(1).index, inplace=True)
            redraw_table()

    def generate_report(b):
        global df
        doc = Document()
        
        # Handle the uploaded Excel file
        if uploader.value:
            uploaded_filename = next(iter(uploader.value))
            uploaded_content = uploader.value[uploaded_filename]['content']
            uploaded_df = pd.read_excel(io.BytesIO(uploaded_content))
            if not uploaded_df.empty:
                doc.add_paragraph("Uploaded Excel Data:")
                table = doc.add_table(rows=1, cols=len(uploaded_df.columns))
                for i, column in enumerate(uploaded_df.columns):
                    table.cell(0, i).text = str(column)
                for i, row in uploaded_df.iterrows():
                    row_cells = table.add_row().cells
                    for j, cell in enumerate(row):
                        row_cells[j].text = str(cell)
                doc.add_paragraph("")  # Add a space between tables
        
        # Add interactive table data to the Word document
        if not df.empty:
            doc.add_paragraph("Interactive Table Data:")
            table = doc.add_table(rows=1, cols=len(df.columns))
            for i, column in enumerate(df.columns):
                table.cell(0, i).text = str(column)
            for i, row in df.iterrows():
                row_cells = table.add_row().cells
                for j, cell in enumerate(row):
                    row_cells[j].text = str(cell)
        
        doc.save('report.docx')
        print("Report generated successfully.")
    
    add_row_button = widgets.Button(description="Add Row")
    add_row_button.on_click(add_row)
    delete_row_button = widgets.Button(description="Delete Row")
    delete_row_button.on_click(delete_row)
    save_button = widgets.Button(description="Save")
    save_button.on_click(generate_report)  # Assuming you meant to save the report here
    generate_report_button = widgets.Button(description="Generate Report")
    generate_report_button.on_click(generate_report)
    
    uploader_box = widgets.VBox([widgets.Label('Upload Excel file:'), uploader])
    buttons_box = widgets.HBox([add_row_button, delete_row_button, save_button, generate_report_button])
    
    display(uploader_box, buttons_box, output)
    redraw_table()

create_interactive_table(df)


Output()